## Задание 1
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

### Решение

In [3]:
import pandas as pd

movies = pd.read_csv('/home/annindark/_pandas1_datasets/ml-latest/movies.csv')
rate = pd.read_csv('/home/annindark/_pandas1_datasets/ml-latest/ratings.csv')

# Вынесла наружу из ф-ции для быстродействия
rate = rate[['movieId','rating']].groupby('movieId').agg('mean')
# соответственно условиям задачи и данным округляем до шага 0.5
rate['rating'] = 0.5*round(rate['rating']/0.5)

def RateType (movieId):
    # Проверяем наличие фильма в списке с рейтингами, ибо иначе ошибка в следующей операции из-за ненайденного индекса
    if any(rate.index == movieId):
        mrate = rate.loc[movieId][0]
        if mrate <= 2:
            return 'low rate'
        elif 2 < mrate <= 4:
            return 'medium rate'
        elif mrate >= 4.5:
            return 'high rate'
    else:
        return None


movies['class'] = movies.movieId.apply(RateType)

## Задание 2

Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:
```
geo_data = {
'Центр': ['москва', 'тула', 'ярославль'],
'Северо-Запад': ['петербург', 'псков', 'мурманск'],
'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}
```
Результат классификации запишите в отдельный столбец region.

### Решение

In [ ]:
import pandas as pd
import re


kv = pd.read_csv('/home/annindark/_pandas1_datasets/ml-latest/keywords.csv')

def GetRegion (keyword):
    if re.search(r'[\b|-]петербур[г|ж]|псков|мурманск]', keyword):
        return 'Северо-Запад'
    elif re.search(r'\bмоск[о]*в|тул|ярослав]', keyword):
        return 'Центр'
    elif re.search(r'\bвладивосто[ч|к]|сахалин|хабаровск]', keyword):
        return 'Дальний Восток'
    else:
        return 'undefined'


kv['region'] = kv.keyword.apply(GetRegion)

## Задание 3 (бонусное)

Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл movies.csv и ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

1. В переменную years запишите список из всех годов с 1950 по 2010.

2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:

- для каждой строки пройдите по всем годам списка years
- если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
- если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год

3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга



### Решение

In [ ]:
import pandas as pd

rate = pd.read_csv('/home/annindark/_pandas1_datasets/ml-latest/ratings.csv')

# Подготовка данных
rate = rate[['movieId','rating']].groupby('movieId').agg('mean')
rate['rating'] = 0.5*round(rate['rating']/0.5)
movies = pd.merge(
    left=rate,
    right=pd.read_csv('/home/annindark/_pandas1_datasets/ml-latest/movies.csv')[['movieId', 'title']],
    how='inner', # нас же не интересуют фильмы без оценок?
    left_on='movieId',
    right_on='movieId')

years = list(range(1950,2011))

def ProductionYear(name):
    # достаем из строки все числа
    namenum = [int(i) for i in ''.join((ch if ch in '0123456789' else ' ') for ch in name).split()]
    # Находим год из справочника years
    # Глазами не нашла примера названия с годом внутри или годом создания не в конце, поэтому такой костыль.
    # В случае фильма с названием 1984 возможна некорректная отработка
    year = list(set(years) & set(namenum))
    if year:
        return year[-1]
    else:
        return 1900


# Обогащаем данные
movies['year'] = movies.title.apply(ProductionYear)

# группируем/усредняем для результата
movies[['year','rating']].groupby('year').agg('mean').sort_values(by='rating', ascending=False)
